In [1]:
import meld_graph
import meld_graph.models
import meld_graph.experiment
import meld_graph.dataset
import meld_graph.data_preprocessing
import meld_graph.evaluation


import importlib
importlib.reload(meld_graph)
importlib.reload(meld_graph.models)
importlib.reload(meld_graph.dataset)
importlib.reload(meld_graph.experiment)
importlib.reload(meld_graph.data_preprocessing)
importlib.reload(meld_graph.evaluation)

import logging
import os
import json

from meld_graph.dataset import GraphDataset, Oversampler
from meld_classifier.meld_cohort import MeldCohort, MeldSubject
from meld_graph.training import Metrics
import numpy as np
from meld_graph.paths import EXPERIMENT_PATH

from meld_graph.evaluation import Evaluator



Setting MELD_DATA_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting BASE_PATH to /rds/project/kw350/rds-kw350-meld/meld_data/Data
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments/co-ripa1/
Setting FS_SUBJECTS_PATH to 
Setting EXPERIMENT_PATH to /rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1


## Define models 

In [2]:
# initialise models you want to run
EXPERIMENT_PATH='/rds/project/kw350/rds-kw350-meld/experiments_graph/kw350'

model_base_paths = {
    # unet deepsupervision trained on big lesions (radius 2)
   'loss_distance_ce_real': '/rds/project/kw350/rds-kw350-meld/experiments_graph/kw350/22-12-11_losses/loss_distance_ce_real/fold_00/',
   'ce_real': '/rds/project/kw350/rds-kw350-meld/experiments_graph/kw350/22-12-11_losses/ce_real/fold_00/',
}

### Create dataset with vals data

In [3]:
model_base_path=model_base_paths['loss_distance_ce_real']
checkpoint_path=os.path.join(EXPERIMENT_PATH, model_base_path)
exp = meld_graph.experiment.Experiment.from_folder(checkpoint_path)

subjects = exp.data_parameters['val_ids']

cohort= MeldCohort(hdf5_file_root='{site_code}_{group}_featurematrix_combat_6.hdf5', dataset='MELD_dataset_V6.csv')

features= exp.data_parameters['features']

# initiate params
params = {                
          "features":features,
          "augment_data": {}, 
          "lesion_bias":0,
          "lobes": False,
          "synthetic_data": {
              'run_synthetic': False,
            },
          "number_of_folds": 10,
          "preprocessing_parameters": {
              "scaling": None, 
              "zscore": '../data/feature_means.json',
            },
          "combine_hemis": None,
          }

#load dataset
dataset = GraphDataset(subjects, cohort, params, mode='val')



Initialised Experiment 22-12-11_losses/loss_distance_ce_real
Loading and preprocessing val data


dataset using distance_maps


Z-scoring data for MELD_H17_3T_FCD_0092
Z-scoring data for MELD_H5_3T_C_0005
Z-scoring data for MELD_H2_15T_C_0002
Z-scoring data for MELD_H4_15T_FCD_0005
Z-scoring data for MELD_H4_3T_FCD_0008
Z-scoring data for MELD_H16_3T_FCD_042
Z-scoring data for MELD_H26_15T_FCD_0006
Z-scoring data for MELD_H4_3T_FCD_0041
Z-scoring data for MELD_H17_3T_FCD_0069
Z-scoring data for MELD_H14_3T_FCD_0028
Z-scoring data for MELD_H3_3T_C_0029
Z-scoring data for MELD_H3_3T_C_0025
Z-scoring data for MELD_H17_3T_FCD_0129
Z-scoring data for MELD_H21_3T_FCD_0022
Z-scoring data for MELD_H15_3T_C_0017
Z-scoring data for MELD_H4_15T_FCD_0022
Z-scoring data for MELD_H24_3T_FCD_0009
Z-scoring data for MELD_H12_3T_FCD_0029
Z-scoring data for MELD_H3_3T_C_0022
Z-scoring data for MELD_H10_3T_FCD_0006
Z-scoring data for MELD_H17_3T_FCD_0037
Z-scoring data for MELD_H3_3T_C_0027
Z-scoring data for MELD_H5_3T_FCD_0016
Z-scoring data for MELD_H17_3T_FCD_0063
Z-scoring data for MELD_H5_3T_FCD_0023
Z-scoring data for MELD

In [16]:
len(dataset.subject_ids)

45

### Load models and predict

In [4]:
use_preload_dataset = True 
output_path = '/rds/project/kw350/rds-kw350-meld/experiments_graph/co-ripa1/22-12-12_evaluation_real_valsdata'

In [5]:
from pyexpat import model

if use_preload_dataset:    
    cohort = None
    subjects = None
else:
    dataset = None

for model_name in model_base_paths.keys(): 
    
    #load experiment already trained using checkpoint path
    model_base_path=model_base_paths[model_name]
    checkpoint_path=os.path.join(EXPERIMENT_PATH, model_base_path)
    exp = meld_graph.experiment.Experiment.from_folder(checkpoint_path)

    # Run the evaluation on the test data and save into directory provided
    save_dir=os.path.join(output_path, model_name)

    eva = Evaluator(experiment = exp,
                    checkpoint_path = checkpoint_path,
                    save_dir = save_dir ,
                    make_images = True,
                    dataset=dataset,
                    cohort=cohort,
                    subject_ids = subjects,
                    mode = 'val'
                )

    # evaluate (predict , stats and plot) or just run individually each step
    # eva.evaluate()
    
    # # load data and predict
    eva.load_predict_data()
    # # calculate stats 
    eva.stat_subjects()
    #  # make images 
    eva.plot_subjects_prediction()

    

Initialised Experiment 22-12-11_losses/loss_distance_ce_real
Creating model
Loading model weights from checkpoint /rds/project/kw350/rds-kw350-meld/experiments_graph/kw350/22-12-11_losses/loss_distance_ce_real/fold_00/best_model.pt
loading data and predicting model
saving prediction for MELD_H17_3T_FCD_0092
saving distance_map for MELD_H17_3T_FCD_0092
saving prediction for MELD_H5_3T_C_0005
saving distance_map for MELD_H5_3T_C_0005
saving prediction for MELD_H2_15T_C_0002
saving distance_map for MELD_H2_15T_C_0002
saving prediction for MELD_H4_15T_FCD_0005
saving distance_map for MELD_H4_15T_FCD_0005
saving prediction for MELD_H4_3T_FCD_0008
saving distance_map for MELD_H4_3T_FCD_0008
saving prediction for MELD_H16_3T_FCD_042
saving distance_map for MELD_H16_3T_FCD_042
saving prediction for MELD_H26_15T_FCD_0006
saving distance_map for MELD_H26_15T_FCD_0006
saving prediction for MELD_H4_3T_FCD_0041
saving distance_map for MELD_H4_3T_FCD_0041
saving prediction for MELD_H17_3T_FCD_0069
s

### Compare models

In [10]:
# load results 
import pandas as pd

df={}
predicted={}
for model_name in model_base_paths.keys():  
    
    results_file = os.path.join(output_path,model_name, 'results', 'test_results.csv')
    df[model_name] = pd.read_csv(results_file) 
    
    

In [11]:
df[model_name]

,ID,group,detected,tp,fp,fn,tn,dice lesional,dice non-lesional
0,MELD_H17_3T_FCD_0092,True,True,163,0,5612,288029,5.490064e-02,0.990352
1,MELD_H5_3T_C_0005,False,False,0,0,0,293804,1.000000e+00,1.000000
2,MELD_H2_15T_C_0002,False,False,0,0,0,293804,1.000000e+00,1.000000
3,MELD_H4_15T_FCD_0005,True,True,1248,0,2127,290429,5.399091e-01,0.996352
4,MELD_H4_3T_FCD_0008,True,False,0,0,1508,292296,6.631300e-19,0.997427
...,...,...,...,...,...,...,...,...,...
95,MELD_H3_3T_C_0002,False,False,0,0,0,293804,1.000000e+00,1.000000
96,MELD_H17_3T_FCD_0109,True,False,0,0,491,293313,2.036660e-18,0.999164
97,MELD_H15_3T_C_0021,False,False,0,0,0,293804,1.000000e+00,1.000000
98,MELD_H11_3T_FCD_0038,True,True,250,74,986,292494,3.205128e-01,0.998191


In [28]:
# stats
import pandas as pd

for model_name in model_base_paths.keys():  
   
    dfsub = df[model_name]    
    predicted = (dfsub['dice_tp']>0)
    print(f'Model {model_name}: \n {predicted.sum()/len(predicted)} sensitivity')

Model unet_deepsuper: 
 0.9382716049382716 sensitivity
Model unet_small_scratch: 
 0.9259259259259259 sensitivity
Model unet_small_finetune: 
 0.9691358024691358 sensitivity
